# 토픽모델링
앞서 워드클라우드를 그렸던 데이터를 이용해 토픽모델링을 수행해 보겠습니다<br>
우선 토픽모델링에 대한 개념설명은 [ratsgo's blog](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/06/01/LDA/)를 참조로 설명<br>
<br>
토픽모델링은 기계학습 알고리즘입니다. 기계가 스스로 설계자인 인간이 미처 파악하지 못하는<br>
추상적인 '토픽'을 스스로 찾아나가는 알고리즘인 것이죠<br>
텍스트 데이터가 풍부할수록 보다 유의미한 결과를 줄 가능성이 높습니다<br>


## 데이터 읽기
여기서는 앞서와 달리 openpyxl이라는 라이브러리를 이용해 엑셀 파일을 여는 느낌으로 파일을 다뤄 보겠습니다

In [ ]:
from openpyxl import load_workbook
from collections import defaultdict
Theme="songpa" # 적절한 이름으로 변경
RawFile='NewsResult_20200229-20200530.xlsx'
wb = load_workbook(RawFile)
ws = wb.active

엑셀 파일을 열어 보면 *키워드*는 *O*열에 위치합니다<br>

In [ ]:
# 키워드(칼럼 O)의 단어들 리스트로 읽기
words = ws['O']
words_list = [x.value.split(',') for x in words[1:]]

전처리는 단순화 해서 전체 텍스트에서 한번 나타난 단어만 제거하는 것으로(빅카인즈 데이터는 사실 전처리가 되어 있는 셈이므로)

In [ ]:
# 한번만 나타난 단어는 제거
frequency = defaultdict(int)
for text in words_list:
    for token in text:
        frequency[token] += 1
words_list = [[token for token in text if frequency[token] > 1] for text in words_list]

## 토픽 모델 만들기
[gensim](https://radimrehurek.com/gensim/)은 토픽모델링을 비롯해 워드엠베딩 등<br> 
다양한 자연어처리 기계학습 알고리즘을 제공하는 매우 유용한 라이브러리입니다<br>
이를 활용해 LDA 모델을 만들어 보겠습니다

In [ ]:
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim import corpora
from gensim.models import LdaModel

### 파라미터 설정
토픽모델링에서 토픽의 숫자는 분석자가 정하는 하이퍼파라미터입니다<br>
해당 말뭉치에 가장 적절한 토픽의 숫자는 몇개일까요?<br>
이것을 추정하는 방법들이 있는데 대표적인 것이 **Perplexity**와 **Coherence** 점수입니다([참조 코어닷 블로그](https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/))<br>
지금은 우선 임의의 숫자로 일단 설정해 보겠습니다

In [ ]:
K = 5 # 일단 임의의 토픽수
iterations = 1000 # 반복 횟수
random_state = 1 # 설정
fname="lda_"+Theme+"K"+str(K)+"R"+str(random_state)+"I"+str(iterations) #임의의 모델명

토픽모델링은 우선 사전을 구축해야 합니다

In [ ]:
dictionary = corpora.Dictionary(words_list)
dictionary.save(fname+'_dictionary.pkl')

그리고 말뭉치를 학습에 적절한 **Bag of Words(BOW) 모델** 형태로 바꾸어 줍니다 

In [ ]:
corpus = [dictionary.doc2bow(text) for text in words_list]
corpora.MmCorpus.serialize(fname+'_corpus.mm', corpus)

이후 실제 모델 구축에 들어갑니다

In [ ]:
lda = LdaModel(corpus, 
               num_topics=K, 
               id2word=dictionary,
               random_state=random_state, 
               iterations=iterations)
lda.save(fname)

### 결과 보기 및 저장
좋은 토픽모델은 각각 모델이 겹치는 단어가 적고<br>
토픽들이 각자 명확한 개성을 나타내는 것입니다<br>
결과가 만족스러운가요?

In [ ]:
lda.print_topics(-1,20)

#### 결과 텍스트 파일로 저장하기

In [ ]:
topics = lda.print_topics(-1,20)
feat_fname = fname+'_feats.txt'
with open(feat_fname, 'w') as text_file:
    for topic_num, features in topics:
        text_file.write("Topic={0} \n {1} \n".format(topic_num, features))

## 시각화
시각화를 해서 보면 보다 명확하게 나타납니다

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [ ]:
vis = pyLDAvis.gensim.prepare(lda,corpus,dictionary,sort_topics=False)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.save_html(vis, fname+'.html')